In [2]:
import json
from pathlib import Path

file_path = Path(r"C:/SKN_3rd/data/youth_policies_api.json"
)

# 파일 읽기
with file_path.open("r", encoding="utf-8") as f:
    data = json.load(f)

In [4]:
import requests
import re
import pickle
import json
import os

def save_json(path: str, file_name: str, data: list):
    """
    데이터를 json 파일로 저장.
    """
    os.makedirs(path, exist_ok=True)
    with open(f"{path}/{file_name}", "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

def load_json(path: str) -> list:
    """
    json 파일에서 데이터 불러오기.
    """
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

In [5]:
# 삭제 항목 정의
remove_keys = [
    "정책 번호", "신청 사이트", 
    "사업관련 참고 사이트 1", "사업관련 참고 사이트 2", "첨부파일"
]

remove_values = [
    "제한없음", "", "-", "상관없음", "□제한없음",
    "□ 제한없음","- 제한없음","-제한없음"
]

# 삭제 함수 정의
def remove_keys_from_data(data, keys):
    if isinstance(data, list):
        return [remove_keys_from_data(item, keys) for item in data]
    elif isinstance(data, dict):
        return {
            key: remove_keys_from_data(value, keys)
            for key, value in data.items()
            if key not in keys
        }
    else:
        return data

def remove_values_from_data(data):
    if isinstance(data, list):
        return [remove_values_from_data(item) for item in data if item not in remove_values]
    elif isinstance(data, dict):
        return {
            key: remove_values_from_data(value)
            for key, value in data.items()
            if value not in remove_values
        }
    else:
        return data


# 삭제
data_cleaned_keys = remove_keys_from_data(data, remove_keys)
preprocess_data = remove_values_from_data(data_cleaned_keys)

# 저장
save_json("../data","policy_result.json", preprocess_data)

print("policy_result.json 저장 완료")

policy_result.json 저장 완료


In [9]:
import json
import re
from collections import Counter

# 불러오기
data = load_json("../data/policy_result.json")

# 모든 텍스트 추출
def text_from_json(data):
    """
    JSON 데이터에서 모든 문자열을 추출.
    """
    texts = []
    if isinstance(data, dict):
        for value in data.values():
            texts.extend(text_from_json(value))
    elif isinstance(data, list):
        for item in data:
            texts.extend(text_from_json(item))
    elif isinstance(data, str):
        texts.append(data)
    return texts

# 단어 분리
def text_to_word(text):
    """
    텍스트를 정제하고 단어 단위로 분리.
    """
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', text)  # 특수문자 제거
    words = text.split()  # 공백 기준으로 단어 분리
    return words

# 자주 등장하는 단어 찾기
def get_frequent_words(words, threshold=100):
    """
    단어 목록에서 자주 등장하는 단어를 찾음.
    """
    word_counts = Counter(words)
    frequent_words = [word for word, count in word_counts.items() if count >= threshold]
    return frequent_words


# 실행
if __name__ == "__main__":

    # 불러오기
    FILE_PATH = "../data/policy_result.json"
    data = load_json(FILE_PATH)
    
    # 텍스트 추출
    texts = text_from_json(data)
    
    # 단어 분리
    all_words = []
    for text in texts:
        all_words.extend(text_to_word(text))
    
    # 50번 이상 등장하는 단어
    threshold = 100
    frequent_words = get_frequent_words(all_words, threshold)
    
    # 출력
    print("[100번 이상 등장하는 단어]")
    for i in range(0, len(frequent_words), 10):
        print(", ".join(frequent_words[i:i + 10]))

[100번 이상 등장하는 단어]
1, 001, 002, 0054002, 0042002, 0044002, 대학생, 다양한, 분야, 등을
위해, 수, 있는, 기회, 제공, 일자리, 취업, 참여자, 모집, 및
등, 20, 월, 1일, 따라, 있음, 시, 사업내용, 또는, 지원
경상남도, N, 0057001, 0056001, 신청방법, 후, 신청, 2025, 12, 예정
별도, 선발, 대상자, 확인서, 0, Y, 0055003, 0043001, 기준, 지역
주소를, 대학, 중인, 자, 3, 단, 경우, 5, 4, 2
따른, 그, 등록, 본인, 제외, 대상, 학생, 기, 6480000, 0011009
0013010, 0014010, 005, 0042006, 보조금, 청년이, 청년, 발굴, 참여권리, 정책인프라구축
지원금액, 활용, 주거, 취창업, 공간, 조성, 프로그램, 운영, 활동, 홍보
간, 제작, 창업, 연계, 모두, 전라남도, 시군, 제출, 제출서류, 사업계획서
공고일, 인정, 서류, 사업, 내, 1차, 서류심사, 2차, 이상, 개인정보
동의서, 발급, 지원사업, 18, 45, 가능, 6460000, 0000000, 0049010, 006
013, 주거비, 전월세, 청년의, 경제적, 주거급여, 최대, 1회, 20250101, 20251231
확인, 지급, 신청서, 이용, 포함, 주민등록등본, 월세, 명의, 0043003, 중위소득
이하, 최근, 6개월, 3개월, 주, 거주하는, 공공기관, 인턴, 미취업, 청년을
0057002, 신청자, 자격증, 관련, 등으로, 이내, 최종, 해당, 불가, 0013003
통해, 취업지원, 서비스, 중소기업, 바랍니다, 지원대상, 111101114011170112001121511230112601129011305113201135011380114101144011470115001153011545115601159011620116501168011710117402611026140261702620026230262602629026320263502

In [10]:
import re

# 불용어 
stopwords = [
    "수행", "경우", "해당", "통하여", "대한", "관련","등", "및", "또는", "중인", "통해",
    "따라", "서비스", "제공", "프로그램", "참여", "따른", "대한", "해당", "관한","이용", 
    "등을", "두고"
]

# 제거 함수
def remove_text(text):
    if isinstance(text, str):
        # URL 제거
        text = re.sub(r'\bhttps?://[^\s]*\.kr\b', '', text)
        
        # 특수기호 제거 (숫자, 한글, 영어 유지)
        text = re.sub(r'[^가-힣a-zA-Z0-9\s.~%]',' ', text)
        
        # 불용어 제거
        for stopword in stopwords:
            text = text.replace(stopword, '')
        return text   
    
    return text

# 데이터 처리
def process_json(data):
    if isinstance(data, dict):
        return {key: process_json(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [process_json(item) for item in data]
    elif isinstance(data, str):
        return remove_text(data)
    else:
        return data


cleaned_data = process_json(data)

# 저장
save_json("../data","policy_result.json",cleaned_data)
print("policy_result.json 저장 완료")

policy_result.json 저장 완료


In [11]:
import json
import re

# 불러오기
data = load_json("../data/policy_result.json")

# 문자열 병합 
def merge_values(item):
    """
    리스트, 딕셔너리, 문자열을 하나의 문자열로 병합
    """
    if isinstance(item, list):
        return " ".join(merge_values(sub_item) for sub_item in item)
    elif isinstance(item, dict):
        return " ".join(f"{key}: {merge_values(value)}" for key, value in item.items())
    elif isinstance(item, str):
        return item.strip()
    else:
        return str(item)
    
# 특정 키는 유지, 나머지는 병합
def restructure_policy_data(data):
    """
    '정책 이름', '기관', '요약', '정책 분야'는 유지하고 나머지는 '내용'에 병합
    """
    result = []
    if isinstance(data, list):
        for item in data:
            if isinstance(item, dict):
                policy = {
                    "정책 이름": item.get("정책 이름", "알 수 없음"),
                    "기관": item.get("기관", "알 수 없음"),
                    "요약": item.get("요약", "알 수 없음"),
                    "정책 분야": item.get("정책 분야", "알 수 없음")
                }

                # '내용'에 나머지 항목 병합
                remaining_content = [
                    f"{key}: {merge_values(value)}"
                    for key, value in item.items()
                    if key not in ["정책 이름", "기관", "요약", "정책 분야"]
                ]
                policy["내용"] = " ".join(remaining_content)

                result.append(policy)
    return result

# 실행행
restructured_data = restructure_policy_data(data)


# 저장
save_json("../data", "policy_result.json", restructured_data)
print("policy_result.json 저장 완료")

policy_result.json 저장 완료


In [12]:
import json
import re

# 불러오기
data = load_json("../data/policy_result.json")

# 지역 매핑 (정규표현식: 기관명에 포함된 지역명)
region_patterns = {
    r'경.*북.*': '경상북도',
    r'경.*남.*': '경상남도',
    r'강.*원.*': '강원도',
    r'전.*북.*': '전라북도',
    r'전.*남.*': '전라남도',
    r'충.*북.*': '충청북도',
    r'충.*남.*': '충청남도',
    r'제.*주.*': '제주특별자치도',
    r'서.*울.*': '서울특별시',
    r'부.*산.*': '부산광역시',
    r'대.*구.*': '대구광역시',
    r'대.*전.*': '대전광역시',
    r'광.*주.*': '광주광역시',
    r'울.*산.*': '울산광역시',
    r'인.*천.*': '인천광역시',
    r'세.*종.*': '세종특별자치시'
}

# 데이터 변환
def replace_institution_with_region(data):
    """
    '기관'을 '지역'으로 키를 대체하고 순서를 유지합니다.
    """
    for policy in data:
        if "기관" in policy:
            institution = policy["기관"]
            region = '전국'  # 기본값 설정
            
            # 기관명에서 지역 매칭
            for pattern, mapped_region in region_patterns.items():
                if re.search(pattern, institution):
                    region = mapped_region
                    break
            
            # '기관'을 '지역'으로 대체 (순서 유지)
            updated_policy = {}
            for key, value in policy.items():
                if key == "기관":
                    updated_policy["지역"] = region
                else:
                    updated_policy[key] = value
            
            # 기존 항목을 새로운 항목으로 교체
            policy.clear()
            policy.update(updated_policy)

# 실행행
replace_institution_with_region(data)

# 저장
save_json("../data", "policy_result.json", data)
print("policy_result.json 저장 완료")

policy_result.json 저장 완료


In [13]:
# 불러오기
data = load_json("../data/policy_result.json")

# 띄어쓰기 제거 
def remove_spaces(data):
    """
    모든 Key에서 띄어쓰기를 제거합니다.
    """
    if isinstance(data, list):
        return [remove_spaces(item) for item in data]
    elif isinstance(data, dict):
        return {key.replace(" ", ""): remove_spaces(value) for key, value in data.items()}
    else:
        return data

# 마지막 2글자 삭제 
def modify_policy_field(data):
    """
    "정책분야"의 Value 마지막 2글자를 삭제합니다.
    """
    if isinstance(data, list):
        for item in data:
            modify_policy_field(item)
    elif isinstance(data, dict):
        if "정책분야" in data and isinstance(data["정책분야"], str):
            data["정책분야"] = data["정책분야"][:-2]  # 마지막 2글자 제거
        for key, value in data.items():
            modify_policy_field(value)

# 실행
data = remove_spaces(data)
modify_policy_field(data)

# 저장
save_json("../data", "policy_result.json", data)
print("policy_result.json 저장 완료")

policy_result.json 저장 완료


In [14]:
from dotenv import load_dotenv

load_dotenv()

True

In [19]:
import os
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_classic.memory import ConversationBufferMemory
from langchain_classic.agents import create_tool_calling_agent, AgentExecutor
from langchain_openai import ChatOpenAI
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory, RunnableLambda
from langchain_classic.schema import HumanMessage
from operator import itemgetter
from textwrap import dedent

In [20]:
# 경로 설정
DIRECTORY_PATH = r"..\data"
PERSIST_DIRECTORY = r"..\data\vector_store\policy"
COLLECTION_NAME = "policy"
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
TOKENIZED_DATA_PATH = os.path.join(DIRECTORY_PATH, "policy_result.json")


# JSON 데이터 불러오기
data = load_json("../data/policy_result.json")

# Document 객체로 변환
documents = []
for policy_name, contents in data.items():
    merged_text = " ".join(contents)
    documents.append(
        Document(
            page_content=merged_text,
            metadata={"name": policy_name, "source": TOKENIZED_DATA_PATH},
        )
    )

AttributeError: 'list' object has no attribute 'items'